## camera variance

In [1]:
import cv2
import numpy as np

dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_250)
parameters =  cv2.aruco.DetectorParameters()

# Camera matrix and distortion coefficients (example values, r""eplace with your own)
"""camera_matrix = np.array([[1000, 0, 640],
                          [0, 1000, 360],
                          [0, 0, 1]], dtype=np.float32)
dist_coeffs = np.zeros((5,))  # Assuming no distortion for simplicity; replace with actual values if available
"""""

camera_matrix = np.array([[1464.9313182647834, 0, 976.9397285664686],
                          [0, 1476.4647299901512, 566.5872185399714],
                          [0, 0, 1]], dtype=np.float32)
dist_coeffs = np.array([- 0.13867638807778296, -0.8932816135659887, -0.0015612679961719675, 0.0012120012424455702, 1.3050427135455158])  # Assuming no distortion for simplicity; replace with actual values if available

# Marker size (in meters)
marker_length = 0.45  # Replace with your marker sizeq

# Arrays to store the translation and rotation data
translations = []
rotations = []

# Open the webcam
cap = cv2.VideoCapture(1)

print("Press 'q' to stop capturing frames and calculate variance.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    # Convert to grayscale for ArUco detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect ArUco markers
    corners, ids, rejected = cv2.aruco.detectMarkers(gray, dictionary, parameters=parameters)

    if ids is not None:
        # Estimate pose of the detected marker
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, marker_length, camera_matrix, dist_coeffs)

        # Draw marker and pose on the frame
        for rvec, tvec in zip(rvecs, tvecs):
            cv2.aruco.drawDetectedMarkers(frame, corners)
            #cv2.aruco.drawAxis(frame, camera_matrix, dist_coeffs, rvec, tvec, 0.1)

            # Store the translation and rotation vectors
            translations.append(tvec[0])  # Extract the translation vector (x, y, z)
            rotations.append(rvec[0])    # Extract the rotation vector (Rodrigues)

    # Display the frame
    cv2.imshow('ArUco Detection', frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

# Convert lists to numpy arrays for easier calculation
translations = np.array(translations)
rotations = np.array(rotations)

# Calculate variance for translation and rotation
translation_variance = np.var(translations, axis=0)
rotation_variance = np.var(rotations, axis=0)

# Display results
print("Translation Variance (x, y, z):", translation_variance)
print("Rotation Variance (Rodrigues x, y, z):", rotation_variance)

print("Translation Variance (x, y, z):", translation_variance*1e6)
print("Rotation Variance (Rodrigues x, y, z):", rotation_variance*(180/np.pi)**2)




Press 'q' to stop capturing frames and calculate variance.


error: Unknown C++ exception from OpenCV code

In [ ]:
Press 'q' to stop capturing frames and calculate variance.
Translation Variance (x, y, z): [3.39999552e-07 3.05553558e-08 2.49620193e-06]
Rotation Variance (Rodrigues x, y, z): [4.79935094e-05 7.04296345e-06 2.23639587e-04]
Translation Variance (x, y, z): [0.33999955 0.03055536 2.49620193]
Rotation Variance (Rodrigues x, y, z): [0.1575534  0.02312069 0.73416546]

## data acquisition

In [4]:
def get_data():
    thymio_data.append({"ground":list(node["prox.ground.reflected"]), 
                        "sensor":list(node["prox.ground.reflected"]),
                        "left_speed":node["motor.left.speed"],
                        "right_speed":node["motor.right.speed"]})
    


## prediction

In [38]:
def prediction_step(X,U,P,previous_t,dt=0.2):

    current_t= time.time()
    if previous_t:
        dt=round(current_t-previous_t,2)
        if dt < 0.2 :
            time.sleep(0.2-dt)
    
    theta=X[2]
    A = np.eye(len(X))
    """B = np.array([0.5*dt*np.cos(theta), 0.5*dt*np.cos(theta)],
                [0.5*dt*np.sin(theta), 0.5*dt*np.sin(theta)],
                [-dt/robot_length,dt/robot_length])"""
    B = np.array([[0.5 * dt * np.cos(theta), 0.5 * dt * np.cos(theta)],
              [0.5 * dt * np.sin(theta), 0.5 * dt * np.sin(theta)],
              [-dt / robot_length, dt / robot_length]])

    x_new= A.dot(X) + B.dot(U)
    Q=np.eye(len(U))*speed_variance
    P += B.dot(Q).dot(B.T) 
    return X,P,current_t
    
    

## update 

In [47]:
def update_step(X,Z,P,aruco_detected):
    R=np.diag([0.6988,0.1634,0.08])
    
    I=Z-X
    H = np.eye(len(X))
    if aruco_detected:
        S=H.dot(P).dot(H.T) + R
    else:
        R_nt=R*10000
        S=H.dot(P).dot(H.T) + R_nt
        
    K_gain = P.dot(H.T).dot(np.linalg.inv(S))
    X = X+ K_gain.dot(I)
    P = P- K_gain.dot(H).dot(P)

    return X, P


## kalman main

In [48]:
def kalman_filter(aruco_detected, x, u, z, P , previous_t):

    X_new, P,next_t  = prediction_step(x, u, P,previous_t)
    X_predicted = X_new
    
    X_new, P = update_step(X_new, z, P, aruco_detected)

    return next_t, X_new, P,X_predicted


In [45]:
import time
robot_length=110
speed_variance = 97.85
X=np.zeros(3)
#U=np.zeros(2)
U =np.array([2, -2], dtype=float)
P=np.eye(3)*1
X=np.zeros(3)
Z = np.zeros(3)

In [49]:
x = np.array([0, 0, 0], dtype=float)
u = np.array([2, -2], dtype=float)
P = np.eye(3) * 1
z = np.array([0, 0, 0], dtype=float)
next_time = 0
for i in range(10):
    z = np.array([2*i, i, i/10], dtype=float)
    next_time, x, P, x_predicted = kalman_filter(True, x, u, z, P, next_time)
    time.sleep(1)
    print(i, np.round(x, 2), np.round(x_predicted,2))

0 [0. 0. 0.] [0. 0. 0.]
1 [1.97 0.46 0.05] [0. 0. 0.]
2 [4.18 1.04 0.12] [1.97 0.46 0.05]
3 [6.58 1.75 0.19] [4.18 1.04 0.12]
4 [9.06 2.62 0.27] [6.58 1.75 0.19]
5 [11.5   3.67  0.35] [9.06 2.62 0.27]
6 [13.76  4.85  0.44] [11.5   3.67  0.35]
7 [15.77  6.1   0.53] [13.76  4.85  0.44]
8 [17.52  7.38  0.63] [15.77  6.1   0.53]
9 [19.04  8.65  0.73] [17.52  7.38  0.63]


In [27]:
next_time = 0
for i in range(10):
    z = np.array([2*i, i, i/10], dtype=float)
    next_time, X, P,X_pred= kalman_filter(True, X, U, Z, P, next_time)
    time.sleep(1)
    print(i, np.round(X, 2), np.round(X_pred,2))

C:\Users\Farah Elsoussi\AppData\Local\Temp\ipykernel_9948\1726262899.py:3: RuntimeWarning: invalid value encountered in multiply
  R_nt=R*np.inf


0 [0. 0. 0.] [0. 0. 0.]
1 [0. 0. 0.] [0. 0. 0.]
2 [0. 0. 0.] [0. 0. 0.]
3 [0. 0. 0.] [0. 0. 0.]
4 [0. 0. 0.] [0. 0. 0.]
5 [0. 0. 0.] [0. 0. 0.]
6 [0. 0. 0.] [0. 0. 0.]
7 [0. 0. 0.] [0. 0. 0.]
8 [0. 0. 0.] [0. 0. 0.]
9 [0. 0. 0.] [0. 0. 0.]


0 [0. 0. 0.] [ 0.    0.   -0.01]
1 [2.  0.5 0.1] [ 0.    0.   -0.07]
2 [4.08 1.14 0.2 ] [2.   0.5  0.03]
3 [6.21 1.95 0.3 ] [4.08 1.14 0.13]
4 [8.33 2.91 0.4 ] [6.21 1.95 0.23]
5 [10.41  4.03  0.5 ] [8.33 2.91 0.33]
6 [12.39  5.28  0.6 ] [10.41  4.03  0.43]
7 [14.25  6.64  0.7 ] [12.39  5.28  0.53]
8 [15.95  8.06  0.8 ] [14.25  6.64  0.63]
9 [17.49  9.5   0.9 ] [15.95  8.06  0.73]
